In [1]:
import pandas as pd 
import requests as req
import json 
import datetime as datetime
import numpy as np
import seaborn as sns
import matplotlib as plt

In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [3]:
json = ('NovDecDataUser.json')

df = pd.read_json(json)

In [4]:
df.tail()

,fullname,id,likes,replies,retweets,text,timestamp,url,user
12376,The Motley Fool,941816438775353344,16,0,9,Say bye-bye to the Easy-Money Fed.https://www....,2017-12-15 23:45:11,/themotleyfool/status/941816438775353344,themotleyfool
12377,CNBC,941817208908398592,16,0,4,"Buy Cisco when it goes below $38, @JimCramer s...",2017-12-15 23:48:15,/CNBC/status/941817208908398592,CNBC
12378,WSJ Markets,941817697825841152,3,0,4,Railroad CSX paid a princely sum to snag an in...,2017-12-15 23:50:11,/WSJmarkets/status/941817697825841152,WSJmarkets
12379,CNBC,941819438638120960,10,5,1,.@JimCramer's fireside chat: Tax reform will '...,2017-12-15 23:57:06,/CNBC/status/941819438638120960,CNBC
12380,CNBC,941820148670795776,10,3,3,Cramer Remix: My biggest worry in this marketh...,2017-12-15 23:59:56,/CNBC/status/941820148670795776,CNBC


In [5]:
df['fullname'].unique()

array(['Jim Cramer', 'WSJ Markets', 'CNBC', 'Financial Times', 'CNNMoney',
       'Justin Wolfers', 'The Motley Fool', 'Bloomberg Markets',
       'Paul Krugman'], dtype=object)

In [6]:
text_sentiment_compound  = []
for i in df['text']:
    compound = analyzer.polarity_scores(i)['compound']
    text_sentiment_compound.append(compound)

In [7]:
df['Compound']=text_sentiment_compound

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12381 entries, 0 to 12380
Data columns (total 10 columns):
fullname     12381 non-null object
id           12381 non-null int64
likes        12381 non-null int64
replies      12381 non-null int64
retweets     12381 non-null int64
text         12381 non-null object
timestamp    12381 non-null datetime64[ns]
url          12381 non-null object
user         12381 non-null object
Compound     12381 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(4), object(4)
memory usage: 1.0+ MB


In [9]:
df['Date']=df['timestamp'].map(lambda x: x.date())

In [10]:
df['UserName'] = df['fullname']

In [11]:
#use the rename(columns={})
df_cleaned_headers = df[['UserName','Date','Compound']]

In [12]:
#get rid of all compounds with a score of equal to zero
df_cleaned_headers = df_cleaned_headers.loc[(df_cleaned_headers['Compound']>0) | (df_cleaned_headers['Compound'] <0)]

In [13]:
df_cleaned_compound = df_cleaned_headers.reset_index(drop=True)

In [14]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import warnings

df_cleaned_compound['Date'] = pd.to_datetime(df_cleaned_compound['Date'])
df_cleaned_compound.head()

,UserName,Date,Compound
0,Jim Cramer,2017-11-15,0.8612
1,Jim Cramer,2017-11-15,0.4199
2,Jim Cramer,2017-11-15,0.7263
3,Financial Times,2017-11-15,0.4939
4,CNNMoney,2017-11-15,-0.8677


In [15]:
df_cleaned_compound = df_cleaned_compound.set_index("Date")

In [20]:
user_dated = df_cleaned_compound.ix['2017-12-01':'2017-12-14']

In [21]:
user_dated.head()

,UserName,Compound
Date,,
2017-12-01,CNNMoney,0.0516
2017-12-01,WSJ Markets,0.0258
2017-12-01,CNBC,-0.5334
2017-12-01,CNBC,0.4404
2017-12-01,Financial Times,0.1280


In [22]:
user_dated.groupby(["UserName"])["Compound"].mean()
user_dated.to_csv("TargetUsers.csv")


In [23]:
user_dated.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3283 entries, 2017-12-01 to 2017-12-14
Data columns (total 2 columns):
UserName    3283 non-null object
Compound    3283 non-null float64
dtypes: float64(1), object(1)
memory usage: 76.9+ KB
